### 1. 모듈

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import time
import pandas as pd
import re

### 2. url 접속

In [2]:
driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

In [3]:
search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)

driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

### 3. 크롤링

In [4]:
num_list = []
handled_list = []
title_list = []
proposer_type_list = []
propose_date_list = []
resolution_date_list = []
resolution_result_list = []
progress_list = []

proposer_list = []
paper_list = []
session_list = []
reason_list = []
board_list = []

In [5]:
n_times = 0
## 원하는 시작페이지
n_iter = 62

driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)').click()
while n_times <= n_iter - 2:
    driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)').click()
    n_times +=1

In [6]:
start = 3
end = 13
## 10페이지가 끝나면 다음 10페이지를 불러옵니다.
while n_iter <= 2810:

    for k in tqdm_notebook(range(start,end)):
        driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({k})').click()
        # 첫 페이지의 의안에 접근합니다.
        for i in range(1,11):
            # i 번쨰 의안에 접근합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i})')

            num = bill.find_element_by_css_selector('td:nth-child(1)').text
            title = bill.find_element_by_css_selector('td.alignL').text
            proposer_type = bill.find_element_by_css_selector('td:nth-child(3)').text
            propose_date = bill.find_element_by_css_selector('td:nth-child(4)').text
            resolution_date = bill.find_element_by_css_selector('td:nth-child(5)').text
            resolution_result = bill.find_element_by_css_selector('td:nth-child(6)').text
            progress = bill.find_element_by_css_selector('td:nth-child(8)').text

            num_list.append(num)
            if resolution_date == '':
                handled_list.append('계')
            else:
                handled_list.append('처')
            title_list.append(title)
            proposer_type_list.append(proposer_type)
            propose_date_list.append(propose_date)
            resolution_date_list.append(resolution_date)
            resolution_result_list.append(resolution_result)
            progress_list.append(progress)


            # 첫 페이지의 i번째 의안에 접속합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i}) > td.alignL > div.pl25 > a')
            bill.click()
            time.sleep(1)

            paper = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(4)').text
            session = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(5)').text     

            paper_list.append(paper)
            session_list.append(session)
            
            ## 제안이유를 담습니다.
            try:
                # 더 보기를 클릭합니다.
                more_info = driver.find_element_by_css_selector('#summaryContentMoreBtn').click()
                reason_raw_text = driver.find_element_by_css_selector('div.contIn > div.textType02').text
                
                content = []
                reason_split = re.split('\r|\t|\n|\xa0', reason_raw_text)
                content.append(''.join(reason_split))
                reason_list.append(content)            
                
            except:
                reason_list.append([]) 

            ## 소관위원회를 담습니다.
            try:
                board = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div:nth-child(5) > div > table > tbody > tr > td:nth-child(1)').text
                board_list.append(board)
            except:
                board_list.append([])
            
            ## 제안자를 담습니다.
            member_list = []    
            try:
                proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
                proposer.click()

                # 새로운 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[1])
                driver.get_window_position(driver.window_handles[1])

                for j in range(2,300):
                    try:
                        member = driver.find_element_by_css_selector(f'#periodDiv > div.layerInScroll.coaTxtScroll > div > a:nth-child({j})').text
                        member_list.append(member)
                        
                    except:
                        break

                driver.close()
                proposer_list.append(member_list)

                # 원래 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[0])
                driver.get_window_position(driver.window_handles[0])

            except:
                proposer_list.append([])

                
            driver.back()
            
    driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({10+start})').click()
    if n_iter == 0:
        start = 3; end = 13
    else :
        start = start; end = end
    n_iter += 1

C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: use driver.switch_to.window instead
C:\Users\NK\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")
C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: DeprecationWarning: use driver.switch_to.window instead


ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=84.0.4147.105)


### 4. 데이터프레임

In [7]:
result = [num_list, handled_list, title_list, proposer_type_list, propose_date_list, resolution_date_list, resolution_result_list,\
          progress_list, paper_list, session_list, reason_list, board_list, proposer_list]
result_df = pd.DataFrame(result).T
result_df.columns = ['의안번호', '처리여부', '의안명', '제안자구분', '제안일자', '의결일자', '의결결과', \
                     '심사진행상태', '문서', '제안회기' , '제안이유', '소관위원회', '제안자']
result_df

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,2021689,처,주택법 일부개정법률안(김재경의원 등 10인),의원,2019-07-26,2020-05-29,임기만료폐기,,의안원문,제20대 (2016~2020) 제369회,[제안이유 및 주요내용감사원의 ‘아파트 층간소음 저감제도 운영실태’ 감사결과에 따르...,국토교통위원회,"[김재경(자유한국당/金在庚), 김무성(자유한국당/金武星), 김석기(자유한국당/金碩基)]"
1,2021688,처,화장품법 일부개정법률안(최도자의원 등 12인),의원,2019-07-26,2020-03-06,수정가결,,의안원문,제20대 (2016~2020) 제369회,[제안이유 및 주요내용정부합동발표(‘16.11) 「생활화학제품안전관리대책」에 따라 ...,보건복지위원회,"[최도자(바른미래당/崔道子), 강훈식(더불어민주당/姜勳植), 권은희(바른미래당/權垠希)]"
2,2021687,처,동물보호법 일부개정법률안(고용진의원 등 12인),의원,2019-07-26,2020-01-09,대안반영폐기,,의안원문,제20대 (2016~2020) 제369회,"[제안이유 및 주요내용현재 시행중인 동물등록제는 유기동물의 발생을 예방하고, 유기동...",농림축산식품해양수산위원회,"[고용진(더불어민주당/高榕禛), 강창일(더불어민주당/姜昌一), 권칠승(더불어민주당/..."
3,2021686,처,폐자원 안전관리 및 주민지원 등에 관한 특별법안(임이자의원 등 11인),의원,2019-07-26,2020-05-20,수정가결,,의안원문\n비용추계서,제20대 (2016~2020) 제369회,[제안이유지난 2018년 11월 수립된 ‘불법폐기물 근절대책’에 따라 정부가 전국의...,환경노동위원회,"[임이자(자유한국당/林利子), 강효상(자유한국당/姜孝祥), 김수민(바른미래당/金秀玟)]"
4,2021685,처,국민체육진흥법 일부개정법률안(김재원의원 등 11인),의원,2019-07-26,2020-05-29,임기만료폐기,,의안원문\n비용추계서 미첨부 사유서,제20대 (2016~2020) 제369회,[제안이유 및 주요내용특별시·광역시·특별자치시·도·특별자치도에 있는 시·도체육회는 ...,문화체육관광위원회,"[김재원(자유한국당/金在原), 강효상(자유한국당/姜孝祥), 경대수(자유한국당/慶大秀)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,2021459,처,지방세특례제한법 일부개정법률안(윤호중의원 등 11인),의원,2019-07-12,2020-01-09,대안반영폐기,,의안원문\n비용추계서,제20대 (2016~2020) 제369회,[제안이유 및 주요내용농수산물의 원활한 유통 및 적정한 가격의 유지를 목적으로 설립...,행정안전위원회,"[윤호중(더불어민주당/尹昊重), 강훈식(더불어민주당/姜勳植), 김현권(더불어민주당/..."
231,2021458,처,‘일본의 對 한국 반도체 소재 수출통제 보복조치’에 대한 조속한 철회 및 사과 ...,의원,2019-07-12,2019-08-09,폐기,,의안원문,제20대 (2016~2020) 제369회,[],외교통일위원회,"[오신환(바른미래당/吳晨煥), 권은희(바른미래당/權垠希), 김관영(바른미래당/金寬永)]"
232,2021457,처,자유무역과 한·일 관계 증진에 반하는 일본 아베 정부의 수출규제 조치 철회 촉구...,의원,2019-07-12,2019-08-09,폐기,,의안원문,제20대 (2016~2020) 제369회,[],외교통일위원회,"[설훈(더불어민주당/薛勳), 고용진(더불어민주당/高榕禛), 권미혁(더불어민주당/權美赫)]"
233,2021456,처,공인회계사법 일부개정법률안(최인호의원 등 10인),의원,2019-07-12,2020-04-29,대안반영폐기,,의안원문,제20대 (2016~2020) 제369회,[제안이유 및 주요내용자격증은 해당 분야의 직무수행에 필요한 지식·기술·소양 등을 ...,정무위원회,"[최인호(더불어민주당/崔仁昊), 강훈식(더불어민주당/姜勳植), 김영춘(더불어민주당/..."


In [ ]:
result_df = result_df[:2100]

In [ ]:
result_df.to_csv('2019_2020_의안정보(6200~).csv')